In [5]:
import requests
from bs4 import BeautifulSoup as BS
import nltk
import urllib.request
import spacy
import textacy
from spacy import displacy
from urllib.request import urlopen
from sys import argv
import wikipedia
import csv
import re
import logging
import gensim
import pkgutil
import sys
from gensim.utils import lemmatize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import pandas as pd

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/faezehghazi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
url = "https://en.wikipedia.org/wiki/EBay"    
html = urlopen(url).read() 
website_url = requests.get('https://en.wikipedia.org/wiki/EBay').text
soup = BS(website_url,'lxml')
#print(soup.prettify())
### Extract the text of the Wikipedia Page
respond = requests.get("https://en.wikipedia.org/wiki/EBay")
ss = BS(respond.text)


Answer your question
Our question was 'What are the most frequent words in the Ebay and how often do they occur?'

### Extract the InfoBox of Ebay
###### Solution1)

In [17]:
import json
tbl = soup.find("table", {"class": "infobox vcard"})

list_of_table_rows = tbl.findAll('tr')
x = {}
for tr in list_of_table_rows:

        th = tr.find("th")
        td = tr.find("td")
        if th is not None:
            innerText = ''
            for elem in td.recursiveChildGenerator():
                if isinstance(elem, str):
                    innerText += elem.strip()
                elif elem.name == 'br':
                    innerText += '\n'
            x[th.text] = innerText
sorted_string = json.dumps(x, indent=4,sort_keys=True)
print(sorted_string)

{
    "Area served": "Worldwide",
    "Founded": "September\u00a03, 1995; 24 years ago(1995-09-03)(Originally asAuctionWeb)",
    "Founder": "Pierre Omidyar",
    "Headquarters": "San Jose, California, U.S.",
    "ISIN": "US2786421030",
    "Industry": "Internet",
    "Key people": "Thomas J. Tierney(Chairman)Scott Schenkel(interimCEO)",
    "Net income": "US$2.530 billion(2018)[1]",
    "Number of employees": "~14,000(December 2018)[1]",
    "Operating income": "US$2.222 billion(2018)[1]",
    "Owner": "Pierre Omidyar(0.0652),The Vanguard Group(0.0621),BlackRock(0.0561)",
    "Revenue": "US$10.746 billion(2018)[1]",
    "Services": "Online shopping",
    "Subsidiaries": "Auction Co.,eBayClassifieds,Kijiji,iBazar,GittiGidiyor,Gumtree,G-Market,StubHub,Half.com,Marktplaats.nl, Qoo10.jp",
    "Total assets": "US$22.819 billion(2018)[1]",
    "Total equity": "US$6.281 billion(2018)[1]",
    "Traded\u00a0as": "NASDAQ:EBAY\nNASDAQ-100 Component\nS&P 500 Component",
    "Type": "Public",
    

### Output Json file

In [21]:
import json
with open('data.json', 'w') as f:
    json.dump(sorted_string, f)

# Extract The Table of Wikipedia Page

Now to extract all the links within , we will use find_all().

In [7]:
table = soup.find('table', attrs={'class': 'wikitable'})  # find the table
Year= []
Revenue = []
Net_income= []
Totall_Asset=[]
Price_Per_Share=[]
Employee=[]
for row in table.find_all('tr')[1:]:  # iterate over the rows, starting from 
    col_0= row.find_all('td')[0]                                 # the second (first one is the header row)
    col_1 = row.find_all('td')[1] #  the Symbol col is the first <td> in every row 
    col_2= row.find_all('td')[2]
    col_3= row.find_all('td')[3]
    col_4= row.find_all('td')[4]
    Year.append(col_0.text)
    Revenue.append(col_1.text)
    Net_income.append(col_2.text)
    Totall_Asset.append(col_3.text)
    Employee.append(col_4.text)
#print(type(tickers))
#Revenue = [i for i in map(lambda s: s.strip(), Revenue)]
Year=list(map(str.strip,Year))
Revenue=list(map(str.strip,Revenue))
Net_income=list(map(str.strip,Net_income))
Totall_Asset=list(map(str.strip,Totall_Asset))
Price_Per_Share=list(map(str.strip,Price_Per_Share))
Employee=list(map(str.strip,Employee))
# ['MMM', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE'

In [8]:
df=pd.DataFrame()
df['Year']= Year
df['Revenue']= Revenue
df['Net_income']= Net_income
df['Totall_Asset']= Totall_Asset
df['Employee']= Employee
df

,Year,Revenue,Net_income,Totall_Asset,Employee
0,2005[55],"4,552","1,082","11,789",15.65
1,2006[56],"5,970","1,126","13,494",13.00
2,2007[57],"7,672",348,"15,366",13.25
3,2008[58],"8,541","1,779","15,592",9.58
4,2009[59],"8,727","2,389","18,408",7.29
5,2010[60],"9,156","1,801","22,004",9.68
6,2011[61],"11,652","3,229","27,320",12.28
7,2012[62],"14,072","2,609","37,074",16.61
8,2013[63],"16,047","2,856","41,488",21.03
9,2014[64],"8,790",46,"45,132",21.01


### Output CSV file

In [13]:
df.to_csv("Ebay_table", sep='|', header=True, index=False) 